# Download dos arquivos de imagem do GeoSampa

In [1]:
import geopandas as gpd
import csv

In [2]:
gdf_scms = gpd.read_file('downloads/SIRGAS_SHP_quadriculaortofoto2017.zip!SIRGAS_SHP_quadriculaortofoto2017/SIRGAS_SHP_quadriculaortofoto2017.shp')

In [3]:
urls = []

for k, i in gdf_scms.iterrows():
    url = f"http://geosampa.prefeitura.sp.gov.br/PaginasPublicas/downloadIfr.aspx?orig=DownloadMapaArticulacao&arq=ORTOFOTOS_2017_RGB%5C{i['qo2010_cod']}.zip&arqTipo=MAPA_ARTICULACAO"
    urls.append([url])

In [4]:
with open('downloads/imagens-2017-url-download.csv', 'w') as f: 
    write = csv.writer(f)
    write.writerows(urls)

## Fazendo Download

In [5]:
# Script independente no arquivo 'download-imagens.py'

## Validando os downloads

In [6]:
import glob
import re

In [7]:
files = glob.glob('downloads/full/*.zip')

In [8]:
scms_baixados = [re.findall(r'[0-9]+-[0-9]+', f)[0] for f in files]

In [9]:
gdf_scms[~gdf_scms['qo2010_cod'].isin(scms_baixados)]['qo2010_cod']

0         2242-3
1         3243-1
2       3335-313
3       2346-434
4       3336-313
          ...   
4245      2222-1
4246      3213-4
4247      3212-2
4248      3215-4
4249      2224-2
Name: qo2010_cod, Length: 4250, dtype: object

In [10]:
urls = []

for i in gdf_scms[~gdf_scms['qo2010_cod'].isin(scms_baixados)]['qo2010_cod'].to_list():
    url = f"http://geosampa.prefeitura.sp.gov.br/PaginasPublicas/downloadIfr.aspx?orig=DownloadMapaArticulacao&arq=ORTOFOTOS_2017_RGB%5C{i}.zip&arqTipo=MAPA_ARTICULACAO"
    urls.append(url)

In [11]:
urls

['http://geosampa.prefeitura.sp.gov.br/PaginasPublicas/downloadIfr.aspx?orig=DownloadMapaArticulacao&arq=ORTOFOTOS_2017_RGB%5C2242-3.zip&arqTipo=MAPA_ARTICULACAO',
 'http://geosampa.prefeitura.sp.gov.br/PaginasPublicas/downloadIfr.aspx?orig=DownloadMapaArticulacao&arq=ORTOFOTOS_2017_RGB%5C3243-1.zip&arqTipo=MAPA_ARTICULACAO',
 'http://geosampa.prefeitura.sp.gov.br/PaginasPublicas/downloadIfr.aspx?orig=DownloadMapaArticulacao&arq=ORTOFOTOS_2017_RGB%5C3335-313.zip&arqTipo=MAPA_ARTICULACAO',
 'http://geosampa.prefeitura.sp.gov.br/PaginasPublicas/downloadIfr.aspx?orig=DownloadMapaArticulacao&arq=ORTOFOTOS_2017_RGB%5C2346-434.zip&arqTipo=MAPA_ARTICULACAO',
 'http://geosampa.prefeitura.sp.gov.br/PaginasPublicas/downloadIfr.aspx?orig=DownloadMapaArticulacao&arq=ORTOFOTOS_2017_RGB%5C3336-313.zip&arqTipo=MAPA_ARTICULACAO',
 'http://geosampa.prefeitura.sp.gov.br/PaginasPublicas/downloadIfr.aspx?orig=DownloadMapaArticulacao&arq=ORTOFOTOS_2017_RGB%5C3335-434.zip&arqTipo=MAPA_ARTICULACAO',
 'http:/

## Descomprimindo os arquivos

In [12]:
import zipfile

In [82]:
for file in files:
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall('downloads/full_unzipped')

In [83]:
## Validando a descompressão

In [13]:
unzipped = glob.glob('downloads/full_unzipped/*.jp2')

In [85]:
scms_unzipped = [re.findall(r'[0-9]+-[0-9]+', f)[0] for f in unzipped]

In [87]:
len(scms_unzipped)

4250

In [88]:
gdf_scms[~gdf_scms['qo2010_cod'].isin(scms_unzipped)]['qo2010_cod']

Series([], Name: qo2010_cod, dtype: object)

## Comprimindo para enviar para o Kaggle

In [1]:
from zipfile import ZipFile
import numpy as np
import glob
import re

In [2]:
imagens = glob.glob('RGB-2017/*')

In [3]:
len(imagens)

8596

In [4]:
np_imagens = np.array(imagens)

In [5]:
np_imagens

array(['RGB-2017/ORTO_3313-133_RGB_1000_v2.jp2',
       'RGB-2017/ORTO_3212-321_RGB_1000_v0.jp2',
       'RGB-2017/ORTO_2322-452_RGB_1000_v2.jp2', ...,
       'RGB-2017/ORTO_3323-212_RGB_1000_v2.jp2',
       'RGB-2017/ORTO_2324-464_RGB_1000_v2.jp2',
       'RGB-2017/Orto_4311-113_RGB_1000_v1.jgw'], dtype='<U46')

In [6]:
scms = np.unique([re.findall(r'[ORTO|Orto]\_[0-9]{3}', f)[0][2:5] for f in np_imagens])

In [7]:
# imagens_list = np.array_split(np.array(imagens), 20)

In [10]:
# glob.glob(f'RGB-2017/*_{431}?-*.*')[0].replace('RGB-2017/', '')


In [9]:
for scm in scms:
   print(scm)
   with ZipFile(f'kaggle/{scm}.zip', 'w') as z:
      # Add  files to the zip
      imagens = glob.glob(f'RGB-2017/*_{scm}?-*.*')
      for i in imagens:
         z.write(f"{i}")

212
222
224
232
234
244
311
321
322
323
324
331
332
333
334
342
343
344
431
432
433
443
444
